#Prediksi Harga Cabai Merah Keriting Pada Provinsi Jawa Timur

# Business Understanding

Proyek ini bertujuan untuk menyediakan prediksi harga cabai merah keriting untuk hari berikutnya secara akurat, sehingga memberikan manfaat signifikan bagi konsumen, petani, dan pedagang.

Konsumen dapat memanfaatkan prediksi ini untuk merencanakan belanja secara lebih efektif, khususnya dalam mengatur anggaran kebutuhan rumah tangga. Dengan mengetahui potensi kenaikan harga, konsumen dapat membeli cabai lebih awal sehingga menghindari pengeluaran yang lebih tinggi dan mengelola keuangan dengan lebih baik.

Petani bisa menggunakan prediksi harga untuk menentukan waktu panen dan distribusi yang optimal agar hasil panen dapat dijual dengan harga terbaik. Informasi ini juga membantu petani meminimalkan kerugian akibat menjual pada saat harga rendah dan meningkatkan efisiensi dalam penyimpanan serta distribusi hasil panen.

Pedagang mendapatkan keuntungan dalam pengelolaan stok dan strategi penetapan harga. Dengan prediksi harga, pedagang dapat menyesuaikan jumlah stok untuk menghindari surplus atau kekurangan cabai, menentukan harga jual yang kompetitif, serta merencanakan promosi yang tepat waktu, seperti memberikan diskon sebelum harga diprediksi naik.

#Data Understanding

data yang saya ambil merupakan data time series univariates yaitu data hagar cabai merah keriting perhari yang diambil dari tanggal 1 januari 2024 sampai 10 oktober 2024 pada provinsi jawa timur dengan jumlah data 464 dan memiliki 2 fitur:tanggal,harga cabai mk
tipe data tanggal adalah time temporal dan harga cabai mk adalah numeric

##menampilkan data

In [ ]:
import pandas as pd

# Membaca file Excel
file_path = '/content/sample_data/Tabel Harga Berdasarkan Daerah (1).xlsx'  # Ganti dengan path file Excel Anda
df = pd.read_excel(file_path)

df.head()

,tanggal,Cabai Merah Keriting
0,02/ 01/ 2023,37350
1,03/ 01/ 2023,38850
2,04/ 01/ 2023,37950
3,05/ 01/ 2023,38150
4,06/ 01/ 2023,39250


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464 entries, 0 to 463
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   tanggal               464 non-null    object
 1   Cabai Merah Keriting  464 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 7.4+ KB


**Deskripsi Data:**

tanggal = merupakan tanggal pada harga cabai merah keriting

Cabai Merah Keriting = merupakan nilai harga dari cabai merah keriting

In [ ]:
df.shape

(464, 2)

Data terdiri dari 464 baris dan 2 kolom

In [ ]:
df.describe()

,Cabai Merah Keriting
count,464.000000
mean,39811.530172
std,13894.865641
min,18500.000000
25%,30275.000000
50%,36350.000000
75%,46050.000000
max,81100.000000


#Data Preprocessing

##Normalisasi Data

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import joblib

# Membaca file Excel
file_path = '/content/sample_data/Tabel Harga Berdasarkan Daerah (1).xlsx'  # Ganti dengan path file Excel Anda
df = pd.read_excel(file_path)

# Inisialisasi MinMaxScaler
scaler = MinMaxScaler()

# Menampilkan nilai minimum dan maksimum sebelum normalisasi
min_values_before = df.min()
max_values_before = df.max()

print("Nilai minimum sebelum normalisasi:")
print(min_values_before)

print("\nNilai maksimum sebelum normalisasi:")
print(max_values_before)
# Melakukan normalisasi pada kolom numerik
kolom_numerik = df.select_dtypes(include=['float64', 'int64']).columns
df[kolom_numerik] = scaler.fit_transform(df[kolom_numerik])

scaler_file_path = '/content/sample_data/scaler.pkl'
joblib.dump(scaler, scaler_file_path)
# Menyimpan data yang sudah dinormalisasi ke file Excel baru
combined_file_path = '/content/sample_data/normalisasi.xlsx'
df.to_excel(combined_file_path, index=False)

Nilai minimum sebelum normalisasi:
tanggal                 01/ 01/ 2024
Cabai Merah Keriting           18500
dtype: object

Nilai maksimum sebelum normalisasi:
tanggal                 31/ 10/ 2023
Cabai Merah Keriting           81100
dtype: object


Disini kami menormalisasi data dulu agar nanti presentase errornya kecil dan kami menggunakan metode minmaxscaler untuk normalisasinya dimana data ini memiliki nilai minimum adalah 18500 dan nilai maksimumnya adalah 81100

##Mencari Outlier

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

# Langkah 1: Baca data dari Excel atau dataset
file_path = '/content/sample_data/normalisasi.xlsx'  # Ganti dengan path file Excel Anda
df = pd.read_excel(file_path)
# Langkah 2: Pilih kolom numerik yang ingin diuji outlier-nya
numerical_columns = ['Cabai Merah Keriting']  # Ganti dengan kolom yang relevan

# Langkah 3: Hitung Z-Score untuk setiap fitur numerik
z_scores = np.abs(stats.zscore(df[numerical_columns]))

# Menentukan threshold, misalnya 3
threshold = 3

# Identifikasi outlier
outliers_zscore = np.where(z_scores > threshold)

# Menampilkan indeks dan kolom outlier
print("Indeks dan kolom yang mengandung outlier (Z-Score):")
print(outliers_zscore)

# Menampilkan data outlier
print("Data outlier berdasarkan Z-Score:")
print(df.iloc[outliers_zscore[0]])


Indeks dan kolom yang mengandung outlier (Z-Score):
(array([], dtype=int64), array([], dtype=int64))
Data outlier berdasarkan Z-Score:
Empty DataFrame
Columns: [tanggal, Cabai Merah Keriting]
Index: []


jadi output diatas memberitahukan bahwa data ini tidak memiliki outlier

##Sliding window

In [ ]:
# Creating lag features for multivariate time series data (PM10 and PM2.5)
# Adding lag of 1 and lag of 2 for both PM10 and PM2.5
file_path = '/content/sample_data/normalisasi.xlsx'  # Ganti dengan path file Excel Anda
data = pd.read_excel(file_path)


data['Cabai Merah Keriting_1'] = data['Cabai Merah Keriting'].shift(1)
data['Cabai Merah Keriting_2'] = data['Cabai Merah Keriting'].shift(2)
data['Cabai Merah Keriting_3'] = data['Cabai Merah Keriting'].shift(3)
data['Cabai Merah Keriting_4'] = data['Cabai Merah Keriting'].shift(4)

data_with_lag = data.dropna()

output_file_path = '/content/sample_data/hasil_sliding_window1.xlsx'  # Ganti dengan path output file Excel
data_with_lag.to_excel(output_file_path, index=False)


karena ini adalah data time series univariate maka kita akan melakukan sliding windows dulu, nah disini kami membagi fiturnya menjadi 4 sehingga data ini memiliki fitur 4 dan targetnya 1

##Pembagian Data Test dan Data Train

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the uploaded file
file_path = '/content/sample_data/hasil_sliding_window1.xlsx'
data = pd.read_excel(file_path)

# Menyiapkan data (X, y)
X = data.drop(columns=['Cabai Merah Keriting', 'tanggal'])
y = data['Cabai Merah Keriting']

# Menyimpan kolom tanggal untuk perbandingan nanti
dates = data['tanggal']

# Split data into training and testing sets
X_train, X_test, y_train, y_test, train_dates, test_dates = train_test_split(
    X, y, dates, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]}")
print(f"Testing set size: {X_test.shape[0]}")
# Simpan data yang dipisah ke dalam satu file Excel dengan sheet berbeda
output_file = '/content/sample_data/bagi.xlsx'
with pd.ExcelWriter(output_file) as writer:
    train_dates,X_train.to_excel(writer, sheet_name='X_train', index=False)
    test_dates,X_test.to_excel(writer, sheet_name='X_test', index=False)
    train_dates,y_train.to_excel(writer, sheet_name='y_train', index=False)
    test_dates,y_test.to_excel(writer, sheet_name='y_test', index=False)


output_file


Training set size: 368
Testing set size: 92


'/content/sample_data/bagi.xlsx'

sebelum melakukan modelling disini kami membagi datanya menjadi data train dan data test yaitu X_train X_test dan y_train y_test, pada data train berjumlah 368 dan data test 92

# Data Modelling

##ensamble bagging

Ensemble Bagging adalah teknik pembelajaran mesin yang digunakan untuk meningkatkan kinerja model dengan menggabungkan prediksi dari beberapa model dasar (base estimators). Istilah "bagging" berasal dari "Bootstrap Aggregating", yang merupakan metode pengambilan sampel untuk menghasilkan variasi dalam model.

Pengertian Bagging
Bootstrap Sampling: Dalam teknik ini, sejumlah subset acak dari data pelatihan diambil dengan pengembalian (replacement). Ini berarti beberapa contoh dalam data pelatihan dapat muncul lebih dari sekali dalam subset, sementara beberapa contoh lainnya mungkin tidak muncul sama sekali.

Model Training: Setiap subset data pelatihan digunakan untuk melatih model dasar yang terpisah. Dalam konteks ini, setiap model dapat berupa algoritma pembelajaran yang berbeda, tetapi sering kali model yang sama digunakan untuk meningkatkan stabilitas.

Prediksi: Setelah semua model dilatih, prediksi dari setiap model dikombinasikan untuk menghasilkan prediksi akhir. Untuk regresi, ini biasanya dilakukan dengan menghitung rata-rata dari semua prediksi, sedangkan untuk klasifikasi, sering kali dilakukan dengan cara pemungutan suara (voting).

Base Estimator: Random Forest Regressor
Random Forest Regressor adalah salah satu jenis model pembelajaran ensemble yang sering digunakan sebagai base estimator dalam teknik bagging. Berikut adalah beberapa fitur utama dari Random Forest Regressor:

Kumpulan Pohon Keputusan: Random Forest terdiri dari banyak pohon keputusan. Setiap pohon dibuat dengan menggunakan subset acak dari data pelatihan dan fitur. Hal ini membuat setiap pohon memiliki ketidakpastian yang berbeda, sehingga dapat menangkap berbagai pola dalam data.

Mengurangi Overfitting: Dengan menggabungkan banyak pohon keputusan, Random Forest cenderung lebih tahan terhadap overfitting dibandingkan dengan satu pohon keputusan. Ini karena kesalahan individual dari pohon-pohon dapat saling menetralkan satu sama lain.

Kekuatan Prediksi: Random Forest Regresor seringkali memberikan hasil yang lebih baik dibandingkan dengan model regresi sederhana karena ia dapat menangkap hubungan yang lebih kompleks dalam data.

In [ ]:
# Mengimpor pustaka yang diperlukan
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
import joblib

scaler_file_path = '/content/sample_data/scaler.pkl'
scaler = joblib.load(scaler_file_path)
# Path file (ganti dengan path file Anda)
file_path = '/content/sample_data/bagi.xlsx'

# Memuat data
data = pd.read_excel(file_path)


# Menginisialisasi model dasar (Regresi Vektor Dukungan)
base_model = LinearRegression()

# Model ensemble Bagging dengan SVR sebagai estimator dasar
bagging_model = BaggingRegressor(estimator=base_model, n_estimators=3, random_state=42)
bagging_model.fit(X_train, y_train)
# Memprediksi pada set pengujian menggunakan ensemble

with pd.ExcelWriter("bagging_samples.xlsx") as writer:
    for i, sample_indices in enumerate(bagging_model.estimators_samples_):
        # Mendapatkan data yang diambil untuk model ke-i berdasarkan indeks
        sampled_data = X_train.iloc[sample_indices].copy()
        sampled_data["Cabai Merah Keriting"] = y_train.iloc[sample_indices].values  # Menambahkan kolom target untuk referensi

        # Menyimpan data ke sheet di file Excel
        sampled_data.to_excel(writer, sheet_name=f"Sample_Model_{i+1}", index=False)

y_pred = bagging_model.predict(X_test)

# Menghitung dan mencetak metrik evaluasi
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"MAE: {mae}, RMSE: {rmse}, MAPE: {mape}")

# --- Memprediksi harga hari berikutnya (11-10-2024) menggunakan 30 data terakhir ---
# Memilih 30 data terakhir untuk prediksi
user_input = {
    "Cabai Merah Keriting_1": 18500,  # Ganti dengan nilai aktual fitur Anda
    "Cabai Merah Keriting_2": 27111,
    "Cabai Merah Keriting_3": 34000,
    "Cabai Merah Keriting_4": 23000  # Sesuaikan jumlah fitur dengan dataset Anda
}

# Konversi input user ke DataFrame
user_input_df = pd.DataFrame([user_input])

user_input_normalized = pd.DataFrame(
    scaler.transform(user_input_df),  # Menggunakan scaler yang dimuat
    columns=user_input_df.columns
)
# Lakukan prediksi
predicted_price = bagging_model.predict(user_input_normalized)
print(f"\nPrediksi harga berdasarkan input user: {predicted_price[0]}")



# Contoh: Asumsikan normalisasi menggunakan Min-Max Scaling
min_harga = 18500  # Nilai minimum harga dalam dataset asli
max_harga = 81100  # Nilai maksimum harga dalam dataset asli

# Denormalisasi hasil prediksi harga untuk semua data dalam array
predicted_next_day_prices_original = predicted_price * (max_harga - min_harga) + min_harga


# Menghitung rata-rata harga prediksi (denormalisasi)
predicted_next_day_price_average = np.mean(predicted_next_day_prices_original)
print(f"\nPrediksi rata-rata harga (denormalisasi) untuk 11-10-2024: Rp {predicted_next_day_price_average:,.2f}")


# --- DataFrame Perbandingan untuk nilai Aktual vs. Prediksi ---
# Membuat DataFrame untuk nilai aktual vs prediksi pada data uji
comparison_df = pd.DataFrame({
    "tanggal": X_test.index,
    "Actual": y_test,
    "Predicted": y_pred
}).sort_values(by="tanggal").reset_index(drop=True)

print("\nHarga Aktual vs Prediksi pada Data Uji:")
print(comparison_df)



MAE: 0.01605198174596433, RMSE: 0.023389096746709224, MAPE: 0.1263255862030535

Prediksi harga berdasarkan input user: -0.02648430610425001

Prediksi rata-rata harga (denormalisasi) untuk 11-10-2024: Rp 16,842.08

Harga Aktual vs Prediksi pada Data Uji:
    tanggal    Actual  Predicted
0         0  0.331470   0.319589
1         9  0.300319   0.304430
2        11  0.304313   0.305674
3        15  0.257188   0.259205
4        19  0.265974   0.270224
..      ...       ...        ...
87      445  0.042332   0.044246
88      447  0.022364   0.025929
89      450  0.027955   0.033692
90      451  0.013578   0.037051
91      454  0.003195   0.014687

[92 rows x 3 columns]


pada model ensamble bagging dengan menggunakan estimator linier regression ini memperlihatkan bahwa model ini memiliki nilai error yang kecil dibandingkan beberapa model yang kami coba yaitu memiliki MAE:0,016 RMSE:0,0233 MAPE:0,126

In [ ]:
# Menyimpan y_pred ke dalam file Excel
output_df = pd.DataFrame({
    "tanggal": X_test.index,
    "Predicted": y_pred
})

# Menyimpan DataFrame ke file Excel
output_path = '/content/sample_data/y_predicted.xlsx'  # Ganti dengan path yang diinginkan
output_df.to_excel(output_path, index=False)

print(f"Hasil prediksi telah disimpan di: {output_path}")


Hasil prediksi telah disimpan di: /content/sample_data/y_predicted.xlsx


##model LTSM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Path file (ganti dengan path file Anda sendiri)
file_path = '/content/sample_data/pembagian.xlsx'

# Memuat data
data = pd.read_excel(file_path)

# Mengubah bentuk X menjadi 3D untuk LSTM [sampel, langkah waktu, fitur]
X_train_reshaped = X_train.values.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_reshaped = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1])

# Mendefinisikan model LSTM
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Melatih model
model.fit(X_train_reshaped, y_train, epochs=100, batch_size=16, validation_split=0.2, verbose=1)

# Membuat prediksi pada set pengujian
y_pred = model.predict(X_test_reshaped)

# Menghitung dan mencetak metrik evaluasi
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"MAE: {mae}, RMSE: {rmse}, MAPE: {mape}")

# --- Prediksi untuk hari berikutnya (11-10-2024) menggunakan 30 data terakhir ---
# Memilih 30 data terakhir dari X
last_30_data = X.iloc[-30:]
last_30_data_reshaped = last_30_data.values.reshape(30, 1, last_30_data.shape[1])

# Memprediksi harga hari berikutnya menggunakan 30 data terakhir dan merata-ratakan prediksi
predicted_next_day_prices = model.predict(last_30_data_reshaped)
predicted_next_day_price = np.mean(predicted_next_day_prices)
print(f"\nPrediksi harga untuk 11-10-2024: {predicted_next_day_price}")

# --- Membandingkan prediksi dengan data aktual ---
# Membuat DataFrame untuk nilai aktual vs prediksi pada data uji
comparison_df = pd.DataFrame({
    "tanggal": X_test.index,
    "Actual": y_test,
    "Predicted": y_pred.flatten()
}).sort_values(by="tanggal").reset_index(drop=True)

print("\nHarga Aktual vs Prediksi:")
print(comparison_df)


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 0.1423 - val_loss: 0.0874
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0866 - val_loss: 0.0395
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0327 - val_loss: 0.0118
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0090 - val_loss: 0.0062
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0073 - val_loss: 0.0053
Epoch 6/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0061 - val_loss: 0.0045
Epoch 7/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0057 - val_loss: 0.0039
Epoch 8/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0053 - val_loss: 0.0034
Epoch 9/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0047 - val_loss: 0.0030
Epoch 10/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0038 - val_loss: 0.0027
Epoch 11/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0037 - val_loss: 0.0026
Epoch 12/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0035 - val_l

##Model SVR

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Path file (ganti dengan path file Anda sendiri)
file_path = '/content/sample_data/pembagian.xlsx'

# Memuat data
data = pd.read_excel(file_path)

# Mendefinisikan dan melatih model SVR
svr_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)  # Parameter dapat disesuaikan sesuai kebutuhan
svr_model.fit(X_train, y_train)

# Memprediksi pada set pengujian
y_pred = svr_model.predict(X_test)

# Menghitung dan mencetak metrik evaluasi
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"MAE: {mae}, RMSE: {rmse}, MAPE: {mape}")

# --- Prediksi untuk hari berikutnya (11-10-2024) menggunakan 30 data terakhir ---
# Mengambil 30 data terakhir untuk prediksi
last_30_data = X.iloc[-30:]

# Memprediksi harga hari berikutnya dengan merata-rata prediksi dari 30 data terakhir
predicted_next_day_prices = svr_model.predict(last_30_data)
predicted_next_day_price = np.mean(predicted_next_day_prices)
print(f"\nPrediksi harga untuk 11-10-2024: {predicted_next_day_price}")

# --- Membandingkan prediksi dengan data aktual ---
# Membuat DataFrame untuk nilai aktual vs prediksi
comparison_df = pd.DataFrame({
    "tanggal": X_test.index,
    "Actual": y_test,
    "Predicted": y_pred
})

# Mengurutkan DataFrame berdasarkan tanggal
comparison_df = comparison_df.sort_values(by="tanggal").reset_index(drop=True)

print("\nHarga Aktual vs Prediksi:")
print(comparison_df)


MAE: 0.04317124124443488, RMSE: 0.05122279276766557, MAPE: 0.6769765920647661

Prediksi harga untuk 11-10-2024: 0.11367973240462349

Harga Aktual vs Prediksi:
    tanggal    Actual  Predicted
0         0  0.331470   0.335119
1         9  0.300319   0.323132
2        11  0.304313   0.322132
3        15  0.257188   0.291776
4        19  0.265974   0.301776
..      ...       ...        ...
87      445  0.042332   0.101750
88      447  0.022364   0.099278
89      450  0.027955   0.099163
90      451  0.013578   0.099668
91      454  0.003195   0.098296

[92 rows x 3 columns]
